<a href="https://colab.research.google.com/github/SEEsuite/colab_scripts/blob/main/vit_classify_from_image_link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Image classification.  

This model architecture (vit) is a recent development in the AI community, and it was trained by the best. However, it only knows how to label 1000 classes, mostly animals, plants, and innoccuous items. You can check the label list [here](https://huggingface.co/google/vit-base-patch16-224/blob/main/config.json). Such a classifier will probably only be useful to you after fine-tuning (additional training on your own data/labels)

I provided two classification models - this model will be more accurate. Use if if you have enough computational resources to handle its size. 




# Use on Your Own Data

In [ ]:
#change here. 
link = "https://docs.google.com/spreadsheets/d/1UZkl8R0963yWcGxno0RZ0DBW0sEx_zZD/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true"

In [ ]:
# importing miscelaneaous packages 
import numpy as np # fast manipulation of multidimensional arrays
from numpy import mean
import pandas as pd

from tqdm.notebook import tqdm as progress_bar # a little vizualization of how fast a loop is running
from scipy.special import softmax
import csv
from datetime import datetime
from matplotlib.dates import date2num

# more packages, tools for getting to google drive
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_data_from_drive(share_link, your_name_for_file="my_data"):
  """Brings data file from a google drive sharepoint to your colab workspace.
     It does not require you to host the dataset on your own account.

     Parameters:
     share_link: the link to view a file in google drive
     our_name_for_file: a string describing the file, preferable endling in a file type, ex. 'data.csv'
     """
  id = share_link.split("/")[5] # separate the id from the link
  print("Using id", id, "to find file on drive")

  # use pydrive and colab modules to authenticate you
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Authenticated colab user")

  # This step will move the file from Drive to the workspace
  download_object = drive.CreateFile({'id':id}) 
  download_object.GetContentFile(your_name_for_file)
  print("Added file to workspace with name", your_name_for_file)

  return

In [ ]:
from datasets import Dataset
# help us manipulate images
from PIL import Image
# helps with API request
import requests

In [ ]:
import_data_from_drive(link, your_name_for_file="insta.xlsx")
df = pd.read_excel('insta.xlsx')

Using id 1ugX3bjxizW2ZdNX5LTBQHxyppyFVPbnq to find file on drive
Authenticated colab user
Added file to workspace with name insta.xlsx


In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Query Id', 'Query Name', 'Date', 'Title',
       'Snippet', 'Full Text', 'Url', 'Domain', 'Sentiment', 'Emotion',
       'Page Type', 'Language', 'Country Code', 'Continent Code', 'Continent',
       'Country', 'Region Code', 'Region', 'City Code', 'Account Type',
       'Assignment', 'Author', 'Avatar', 'Category Details', 'Checked', 'City',
       'Display URLs', 'Expanded URLs', 'Facebook Author ID',
       'Facebook Comments', 'Facebook Likes', 'Facebook Role',
       'Facebook Shares', 'Facebook Subtype', 'Full Name', 'Gender',
       'Hashtags', 'Impact', 'Impressions', 'Instagram Comments',
       'Instagram Followers', 'Instagram Following', 'Instagram Likes',
       'Instagram Posts', 'Interest', 'Last Assignment Date', 'Latitude',
       'Location Name', 'Longitude', 'Media Filter', 'Media URLs',
       'Mentioned Authors', 'Priority', 'Professions', 'Resource Id',
       'Short URLs', 'Starred', 'Status', 'Subtype', 'Tags', 'Thread Autho

In [ ]:
import json


In [ ]:
def app(url):
  return url.strip(']\'[').split(', ')
df["temporary_urls"] = df["temporary_urls"].apply(app)

In [ ]:
def to_pillow(batch):
    """ a transformation to be applyed to a dataset. takes in a url string and returns data with a new column"""
    images = []
    for url_list in batch['temporary_urls']:
      # print(url_list)
      for url in url_list: # I need to get this running for multiple images per post
        # print(url)
        try:
          http_image = requests.get(url, stream=True).raw #1
          image = Image.open(http_image) # step 2
          # print(type(http_image))
          # image.show()
          # print(type(image))
        except Exception as e:
          print(e)
          image = None
        images.append(image)
    batch['images'] = images # will probably need to expand this out in practice
    return batch

In [ ]:
data = Dataset.from_pandas(df)
data.set_transform(to_pillow)

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,Query Id,Query Name,Date,Title,Snippet,Full Text,Url,Domain,...,Reddit Score,Reddit Score Upvote Ratio,Reddit Author Karma,Reddit Author Awardee Karma,Reddit Author Awarder Karma,Reddit Comments,Subreddit,Subreddit Subscribers,temporary_urls,final_urls
0,0,1.0,2.001569e+09,Sponsored IG test,2023-01-01 00:50:40.000,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,https://www.instagram.com/p/Cm2mIrYrUJb/,instagram.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-2.cdninstagram.com/v/t...,[https://scontent-iad3-2.cdninstagram.com/v/t5...
1,2,3.0,2.001569e+09,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-2.cdninstagram.com/v/t...,[https://scontent-iad3-2.cdninstagram.com/v/t5...
2,3,4.0,2.001569e+09,Sponsored IG test,2022-12-31 14:35:25.000,🍾𝐍𝐞𝐰 𝐘𝐞𝐚𝐫'𝐬 𝐄𝐯𝐞 𝐁𝐨𝐨𝐤 𝐅𝐞𝐚𝐭𝐮𝐫𝐞🍾 ⠀ 𝘛𝘩𝘢𝘯𝘬 𝘺𝘰𝘶 {har...,...#gnomesofinstagram #bookphotography #northc...,🍾𝐍𝐞𝐰 𝐘𝐞𝐚𝐫'𝐬 𝐄𝐯𝐞 𝐁𝐨𝐨𝐤 𝐅𝐞𝐚𝐭𝐮𝐫𝐞🍾 ⠀ 𝘛𝘩𝘢𝘯𝘬 𝘺𝘰𝘶 {har...,https://www.instagram.com/p/Cm1fufQL0zR/,instagram.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-1.cdninstagram.com/v/t...,[https://scontent-iad3-1.cdninstagram.com/v/t5...
3,5,6.0,2.001569e+09,Sponsored IG test,2022-12-30 13:18:36.000,✨BOOK FEATURE✨ ✨Happy Friday bookworms! I’m su...,...#harlequinpublicityteam #harlequinbooks #ca...,✨BOOK FEATURE✨ ✨Happy Friday bookworms! I’m su...,https://www.instagram.com/p/CmyyI8crypt/,instagram.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-1.cdninstagram.com/v/t...,[https://scontent-iad3-1.cdninstagram.com/v/t5...
4,6,7.0,2.001569e+09,Sponsored IG test,2022-12-30 04:22:01.000,USE MY CODE: ElijahsInsanity or click the link...,USE MY CODE: ElijahsInsanity or click the link...,USE MY CODE: ElijahsInsanity or click the link...,https://www.instagram.com/p/Cmx0u10u_n1/,instagram.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-2.cdninstagram.com/v/t...,[https://scontent-iad3-2.cdninstagram.com/v/t5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,1036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-2.cdninstagram.com/v/t...,[https://scontent-iad3-2.cdninstagram.com/v/t5...
1298,1041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-1.cdninstagram.com/v/t...,[https://scontent-iad3-1.cdninstagram.com/v/t5...
1299,1042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-2.cdninstagram.com/v/t...,[https://scontent-iad3-2.cdninstagram.com/v/t5...
1300,1045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-iad3-2.cdninstagram.com/v/t...,[https://scontent-iad3-2.cdninstagram.com/v/t5...


In [ ]:
import math
from PIL import Image, ImageDraw
  

In [ ]:
from PIL import Image
import torch
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
from datasets import load_dataset


# processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
# model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")


from transformers import AutoFeatureExtractor, AutoModelForImageClassification

processor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
#Just copy and pasted from image_analysis_1!
predicted_labels = []
model.to('cuda')
for d in progress_bar(data): # change 1

  #pass images through the preprocessor
  texts = [[ "a cat", "a dog", "a closeup photo of a persons face", "a book", "a poster", "a group of people laughing together", "a bench", "a cup", "a cup held in someone's hand",  "a hand with manicured nails", 'a photo containing the name "kit coltraine"']]
  images = d['images']

  # print(len(image))

  if images:
    inputs = processor(images, return_tensors="pt").to('cuda')# change 2
  else:
    predicted_labels.append("")
    continue

  with torch.no_grad():
    logits = model(**inputs).logits

  # model predicts one of the 1000 ImageNet classes
  predicted_label = logits.argmax(-1).item()
  our_label = model.config.id2label[predicted_label]
  predicted_labels.append(our_label)





  0%|          | 0/1302 [00:00<?, ?it/s]

In [ ]:

df["predicted_label"] = predicted_labels
df.to_excel("my_data_with_class_label.xlsx")